<a href="https://colab.research.google.com/github/naomori/codexa_LogisticRegression_Practice/blob/master/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Section.1: ロジスティック回帰のモデル

0-1の値をモデル化することにより、データセット内の観測値が
特定のクラスに属する確率を本質的に与えてくれる。

ロジスティック回帰では、直線ではなく、0-1のS字曲線を使う。

## Section.2: シグモイド関数（Sigmoid Function）

シグモイド関数は、値を0-1のレン時にコンパクトにする計算処理を行う。

$\frac{e^{(w_1x+w_0)}}{1+e^{(w_1x+w_0)}}$

あるいは、

$\frac{1}{1+e^{-(w_1x+w_0)}}$

* $w_1$ は係数または傾き
* $w_0$ は定数項またはY切片
* $x$ はデータ内の特徴量

$y = w_1x + w_0$ は線形回帰のモデル式であり、  
シグモイド関数は、このモデル式を圧縮して０～１の確率となる値を出力する。

$e$はネイピア数で Napier's constant または オイラー数（Euler's constant）



In [3]:
import numpy as np
print(np.e)

2.718281828459045


## Section.3: 特徴量１つのロジスティック回帰



In [0]:
from google.colab import files
uploaded = files.upload() 

In [0]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import LabelEncoder

In [0]:
# Pandas の表示設定
pd.set_option('max_columns', 50)
pd.set_option('max_rows', 500000)

In [0]:
# CSVファイルの読み込み
mushroom = pd.read_csv('mushrooms.csv')

In [9]:
# データセットの最初の５行を表示
mushroom.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


class が今回のターゲットであり、p(poison)=毒性, e(edible)=食用 の２種類の値を持っている。

今回は特徴量として、 gill-size（ひだのサイズ）と bruises（あざ）のみを使う。

In [10]:
# データセットの切り分け
mushroom = mushroom[['class', 'gill-size', 'bruises']]
mushroom.head()

,class,gill-size,bruises
0,p,n,t
1,e,b,t
2,e,b,t
3,p,n,t
4,e,b,f


### 1. ダミー変数へ変換


データフレームの値はすべて文字列になっている。
このままではロジスティック回帰としての処理ができないため、
それぞれの値に属している場合を「1」、属していない場合を「0」というダミー変数へ変換する。

In [14]:
# カラム情報を引き継いで、ダミー変数へ変換した mushroom2 を作成
mushroomcol = mushroom.columns[1:]
print(mushroomcol)
mushroom2 = pd.get_dummies(mushroom, columns=mushroomcol, drop_first=True)
mushroom2.head()

Index(['gill-size', 'bruises'], dtype='object')


,class,gill-size_n,bruises_t
0,p,1,1
1,e,0,1
2,e,0,1
3,p,1,1
4,e,0,0


### 2. 文字列から数値へ変換

次に class の値を毒性「1」、食用「0」と数値に変換する。

In [0]:
# LabelEncoder関数を使い「class」の文字列を数値へ変換
labelencoder = LabelEncoder()
mushroom2['class'] = labelencoder.fit_transform(mushroom2['class'])

In [16]:
# データフレームの確認
print(mushroom2.shape)
mushroom2.head()

(8124, 3)


,class,gill-size_n,bruises_t
0,1,1,1
1,0,0,1
2,0,0,1
3,1,1,1
4,0,0,0


これで、データの前処理が完了した。

bruises（あざ）の有無を特徴量として、キノコの「class」を予測するロジスティック回帰のモデルを作ってみる。

In [17]:
# statsmodel を利用してロジスティック回帰のモデルを構築する
mushroom2 = sm.add_constant(mushroom2)
logit = sm.Logit(mushroom2['class'], mushroom2[['const', 'bruises_t']])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.559154
         Iterations 5


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [18]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                 8124
Model:                          Logit   Df Residuals:                     8122
Method:                           MLE   Df Model:                            1
Date:                Wed, 31 Jul 2019   Pseudo R-squ.:                  0.1926
Time:                        07:01:18   Log-Likelihood:                -4542.6
converged:                       True   LL-Null:                       -5625.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8158      0.031     25.920      0.000       0.754       0.877
bruises_t     -2.2997      0.054    -42.295      0.000      -2.406      -2.193
==============================================================================
"""

In [19]:
# 10番目のサンプルデータを表示

mushroom2[mushroom2.index == 10]

,const,class,gill-size_n,bruises_t
10,1.0,0,0,1


シグモイド関数を使ってみる。

$\frac{e^{(w_1x+w_0)}}{1+e^{(w_1x+w_0)}}$

予測したい10番目のキノコの bruises の値は「1」なので、$x = 1$

係数（coefficient）である $w_1$ の値は、-2.2997 

定数項（constatnt）である $w_0$ の値は、0.8158

In [22]:
import numpy as np
model1 = np.exp(.8158 + (-2.2997*1)) / (1.0 + np.exp(.8158 + (-2.2997*1)))
print('Index 10 has a', round(model1 * 100, 2), '% probability of being posonnous')

Index 10 has a 18.48 % probability of being posonnous


一般的なロジスティック回帰では、出力された確率が 50 %以上で分類を行う。

今回の結果は 18.48 % と 50% 以下なので、食用となる。
実際の10番目のキノコのクラスは 0（食用）なので正しい。

## Section.4: 複数の特徴量でのロジスティック回帰

複数の特徴量に適用するシグモイド関数は以下の通り。

$\frac{e^{(w_0+w_1x_1+..w_nx_n)}}{1 + e^{(w_0+w_1x_1+..w_nx_n)}}$



In [23]:
# 特徴量2つのロジスティック回帰モデルの作成

logit = sm.Logit(mushroom2['class'], mushroom2[['const', 'bruises_t', 'gill-size_n']])
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.463890
         Iterations 6


In [24]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  class   No. Observations:                 8124
Model:                          Logit   Df Residuals:                     8121
Method:                           MLE   Df Model:                            2
Date:                Wed, 31 Jul 2019   Pseudo R-squ.:                  0.3301
Time:                        07:16:54   Log-Likelihood:                -3768.6
converged:                       True   LL-Null:                       -5625.9
Covariance Type:            nonrobust   LLR p-value:                     0.000
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0046      0.038      0.121      0.904      -0.070       0.079
bruises_t      -1.9086      0.060    -31.721      0.000      -2.027      -1.791
gill-size_n     2.5106      0.073     34.363      0.000       2.367       2.654
===============================================================================
"""

In [25]:
mushroom2[mushroom2.index == 10]

,const,class,gill-size_n,bruises_t
10,1.0,0,0,1


In [0]:
w_0 = 0.0046
w_1 = -1.9086
w_2 = 2.5106
x_1 = 1
x_2 = 0

In [27]:
e_cal = np.exp(w_0 + w_1*x_1 + w_2*x_2)
model2 = e_cal / (1.0 + e_cal)
print('Index 10 has a', round(model2 * 100, 2), '% probability of being poisonus')

Index 10 has a 12.97 % probability of being poisonus
